Assignment 3 - Your Project Database

Due Sunday, November 5, 2017

Submission: Put the data and Jupyter notebook files in a folder. Make sure all links to data are relative to the folder so the TAs can run the notebooks.

You MUST use the data set that you will use in your project.
In this assignment, you will create a database for analysis for your project. The assignment can be done together for group projects.

Part A - Create a database (30 points)
Create a database for your project. If a relational database it must be in at least third normal form. Your database can be SQL, NoSQL or Hadoop Distributed File System (HDFS). Show your schema.

Part B - Queries (30 points)
Generate a list of five common queries for your data.
Implement the five common queries.
Time the implementation of the five common queries and give an estimate of how they scale.

Part C - Back-up your data on Google Cloud (40 points)
Create a scheme to back-up your data.
Implement your scheme to back-up your data on Google Cloud. The database itself can also be on Google Cloud.

In [1]:
###In the below code we are trying to mine texts from common crwaller website based on specific domain
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io
from bs4 import BeautifulSoup

import sys
#reload(sys)
#sys.setdefaultencoding('utf8')

##Input texts
domain = ['https://unsupervisedmethods.com','http://deeplearning.net/','https://machinelearningmastery.com','aitreands.com','sciendedaily.com','machinelarningmastery.com','news.mit.edu','aitopics.com','mlweekly.com','machinelearning.co','artificial-intelligence.blog','chatbotsmagazine.com']

##list of indices to search from common crawl website
index_list = ['2017-43','2017-39','2017-34','2017-30','2017-26','2017-22','2017-17','2017-13','2017-09','2017-04']


In [2]:
#Search for common crawl index for a domain
def search_domain(domain):
    record_list=[]
    
    print("[*] trying target domain {0}".format(domain))
    
    for index in index_list:
        cc_url = "http://index.commoncrawl.org/CC-MAIN-{0}-index?".format(index)
        cc_url += "url={0}&output=json".format(domain)
        
        response = requests.get(cc_url)
        print(response)
        if response.status_code == 200:
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added {0} results.".format(len(records)))
        
    print("[*] Found a total of {0} hits.".format(len(record_list)))
    
    return record_list

In [3]:
##fetch data from the page found

def download_data(record):
    
    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1
    
    ##Getting data from s3 aws.com
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    #We use range header to just ask this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={0}-{1}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    
    print(prefix + record['filename'])
    
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    if f:
        data = f.read()
        #print(data)
        response = ""
        #print(data)
        data_string = data.decode('utf-8')
        if len(data_string):
            try:
                warc,header,response=data_string.strip().split('\r\n\r\n',2)
            except:
                pass
    return response

In [4]:
##Search for domain and extract raw HTML
from bs4.element import Comment
import urllib

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def extract_html(html_content,link_list):
    parser = BeautifulSoup(html_content)
    links = parser.find_all('a')
    if links:
        for link in links:
            href = link.attrs.get("href")
            if href is not None:
                
                if domains not in href:
                    if href not in link_list and href.startswith("http"):
                        #print("[*] Discovered external link: {0}".format(href))
                        link_list.append(href)

    texts = parser.findAll(text=True)
    visible_texts = filter(tag_visible, texts) 
    for t in visible_texts:
        t = t.strip()
        if t:
            link_list.append(t)
                
                       
    return link_list

In [5]:
link_list = []
for domains in domain:
    record_list = search_domain(domains)
    for record in record_list:
        html_content = download_data(record)
        print("[*] Retrieved {0} bytes for {1}".format(len(html_content),record['url']))
        link_list = extract_html(html_content,link_list)
    
        #print("[*] Total external links discovered: {0}".format(len(link_list)))
        #try:
         #   with io.open("WebScrapper-texts.csv","a+") as output:
    
          #      fields = ["URL"]
    
           #     logger = csv.DictWriter(output,fieldnames=fields)
           #     logger.writeheader()
        
            #    for link in link_list:
             #       logger.writerow({"URL": link})
        #except:
         #   pass
          

[*] trying target domain https://unsupervisedmethods.com
<Response [200]>
[*] Added 2 results.
<Response [404]>
<Response [200]>
[*] Added 2 results.
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
[*] Found a total of 4 hits.
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-43/segments/1508187826642.70/crawldiagnostics/CC-MAIN-20171023202120-20171023222120-00540.warc.gz
[*] Retrieved 176 bytes for http://unsupervisedmethods.com/


C:\Users\Tushar\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Tushar\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-43/segments/1508187826642.70/crawldiagnostics/CC-MAIN-20171023202120-20171023222120-00515.warc.gz
[*] Retrieved 152 bytes for https://unsupervisedmethods.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-34/segments/1502886105086.81/crawldiagnostics/CC-MAIN-20170818175604-20170818195604-00360.warc.gz
[*] Retrieved 176 bytes for http://unsupervisedmethods.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-34/segments/1502886105086.81/crawldiagnostics/CC-MAIN-20170818175604-20170818195604-00695.warc.gz
[*] Retrieved 152 bytes for https://unsupervisedmethods.com/
[*] trying target domain http://deeplearning.net/
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Added 1 results.
<Response [200]>
[*] Adde

https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-04/segments/1484560279169.4/warc/CC-MAIN-20170116095119-00289-ip-10-171-10-70.ec2.internal.warc.gz
[*] Retrieved 36947 bytes for http://machinelearningmastery.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-04/segments/1484560280483.83/warc/CC-MAIN-20170116095120-00289-ip-10-171-10-70.ec2.internal.warc.gz
[*] Retrieved 36991 bytes for http://machinelearningmastery.com/
[*] trying target domain aitreands.com
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
[*] Found a total of 0 hits.
[*] trying target domain sciendedaily.com
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
[*] Found a total of 0 hits.
[*] trying target domain machinelarningmastery.com
<Response [404]>
<R

https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-17/segments/1492917120187.95/warc/CC-MAIN-20170423031200-00049-ip-10-145-167-34.ec2.internal.warc.gz
[*] Retrieved 119058 bytes for http://news.mit.edu/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-17/segments/1492917120878.96/warc/CC-MAIN-20170423031200-00049-ip-10-145-167-34.ec2.internal.warc.gz
[*] Retrieved 114182 bytes for http://news.mit.edu/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-17/segments/1492917121644.94/warc/CC-MAIN-20170423031201-00630-ip-10-145-167-34.ec2.internal.warc.gz
[*] Retrieved 118789 bytes for https://news.mit.edu/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-17/segments/1492917121865.67/warc/CC-MAIN-20170423031201-00049-ip-10-145-167-34.ec2.internal.warc.gz
[*] Retrieved 111887 bytes for http://news.mit.edu/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-17/segments/1492917121869.65/warc/CC-MAIN-20170423031201-00049-ip-10-145-167-34.ec

https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-26/segments/1498128322275.28/warc/CC-MAIN-20170628014207-20170628034207-00331.warc.gz
[*] Retrieved 24519 bytes for http://mlweekly.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-22/segments/1495463608870.18/warc/CC-MAIN-20170527055922-20170527075922-00459.warc.gz
[*] Retrieved 22440 bytes for http://mlweekly.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-17/segments/1492917122621.35/warc/CC-MAIN-20170423031202-00191-ip-10-145-167-34.ec2.internal.warc.gz
[*] Retrieved 23633 bytes for http://mlweekly.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-13/segments/1490218189032.76/warc/CC-MAIN-20170322212949-00136-ip-10-233-31-227.ec2.internal.warc.gz
[*] Retrieved 23406 bytes for http://mlweekly.com/
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-09/segments/1487501170624.14/warc/CC-MAIN-20170219104610-00155-ip-10-171-10-108.ec2.internal.warc.gz
[*] Retrieved 24

# Part A - Creation of Database

We are using MongoDB local database server, in which we store the data

In [7]:
import pymongo

# Connection to Mongo DB
try:
    conn=pymongo.MongoClient()
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print("Could not connect to MongoDB: {0}").format(e)
conn

Connected successfully!!!


MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [8]:
db = conn['textMining']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'textMining')

Create Database

In [10]:
db = conn.text_mining

In [11]:
text_table = db.text_table

# Part B - Execution of common queries and their execution time

In [ ]:
Insert Command

In [12]:
for link in link_list:
    data = {}
    data['text/url'] = link
    #insert data
    text_table.insert_one(data)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

data

In [ ]:
text_table.count()

In [ ]:
#Find 2 rows from last
for d in text_table.find()[:2]:
    print (d)

In [ ]:
#Find count of rows based on query
text_table.find({'text/url':'301 Moved Permanently'}).count()

In [ ]:
text_table.find({'text/url':'301 Moved Permanently'}).explain()

In [ ]:
#Update command
text_table.insert({'_id':12345,'text/url':'301'})

In [ ]:
text_table.update({'_id':1234},{'text/url':'URL'})

text_table.find_one({'_id':1234})

In [ ]:
conn.database_names()

In [ ]:
text_table.remove({'_id':1234})